In [1]:
import argparse
import subprocess
from typing import TypedDict , Optional
import matplotlib.pyplot as plt
import decimal
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
import multiprocessing
import concurrent

In [2]:

from enum import Enum


base_dir = "/opt/noxim"
bin_path = f"{base_dir}/bin/noxim -power {base_dir}/bin/power.yaml -config {base_dir}/config_examples/default_config.yaml"

class Routing(Enum):
    XY = "XY"
    DELTA = "DELTA"
    WEST_FIRST = "WEST_FIRST"
    NORTH_LAST = "NORTH_LAST"
    NEGATIVE_FIRST = "NEGATIVE_FIRST"
    ODD_EVEN = "ODD_EVEN"
    DYAD = "DYAD"
    TABLE_BASED = "TABLE_BASED"

class Traffic(Enum):
    RANDOM = "random"
    TRANSPOSE1 = "transpose1"
    TRANSPOSE2 = "transpose2"
    HOTSPOT = "hostspot"
    BIT_REVERSAL = "bitreversal"
    SHUFFLE = "shuffle"
    BUTTERFLY = "butterfly"

class Res(TypedDict):
    total_received_packets: int
    average_delay: float
    network_throughput: float
    total_energy: float


class CMD(TypedDict):
    load: float
    size : [int , int]
    packet : int
    routing : Optional[Routing]
    traffic : Optional[Traffic]
    buffer : Optional[int]
    vs : Optional[int]

def make_cmd(load, 
            size : [int ,int] ,
            traffic: Optional[Traffic] = None ,
            routing : Optional[Routing] = None ,
            buffer :Optional[int] = None  ,
            packet :int = 8 ,
            vc :Optional[int] = None ) -> CMD:
    cmd : CMD = { "load" : load , "routing" : routing , "size" : size , "traffic" : traffic  , "buffer" : buffer , "packete" : packet , "vc" : vc}
    return cmd

def cmd_str(cmd : CMD) -> str:
    [load , size , packet , routing , traffic , buffer_size , vc_count] = [cmd['load'] , cmd['size'] , cmd['packete'] , cmd['routing'] , cmd['traffic'] , cmd['buffer'] , cmd['vc']]
    str = ""
    str += f"-pir {load / packet} poisson -dimx {size[0]} -dimy {size[1]} -size {packet} {packet} "
    if routing is not None:
        str += f"-routing {routing.value} "
    if traffic is not None:
        str += f"-traffic {traffic.value} "
    if buffer_size is not None:
        str += f"-buffer {buffer_size} "
    if vc_count is not None:
        str += f"-vc {vc_count} "

    return str


def parse_noxim_output(output : str) -> Res:
    data : Res = {}
    lines = output.split('\n')
    for line in lines:
        if 'Total received packets' in line:
            data['total_received_packets'] = int(line.split(': ')[1])
        elif 'Global average delay (cycles)' in line:
            data['average_delay'] = float(line.split(': ')[1])
        elif 'Network throughput (flits/cycle)' in line:
            data['network_throughput'] = float(line.split(': ')[1])
        elif 'Total energy (J)' in line:
            data['total_energy'] = float(line.split(': ')[1])
    return data



In [17]:
def run_nox(cmd : CMD , parser = parse_noxim_output ,bin = bin_path) -> Res  :
    str_cmd = f"{bin} {cmd_str(cmd)}" 
    result = subprocess.run(str_cmd, shell=True, capture_output=True, text=True)
    parsed = parser(result.stdout)
    return [cmd,parsed]


def frange(x, y, jump):
  while x < y:
    yield x
    x += jump

type frng = { "start": float , "end" : float , "step" : float }

# def runOnRange(routing : Routing , size : [int , int] , traffic : Traffic , rng : frng):
#     results = []
#     loads = [x for x in frange(rng["start"], rng["end"], rng["step"])]
#     for load in tqdm(loads):
#         cmd = make_cmd(load, routing, size, traffic)
#         output = run_nox(cmd)
#         results.append([cmd["load"] , output[1]])
#     return results

def runOnRange_par(cmd : CMD , rng : frng):
    results = []
    loads = [x for x in frange(rng["start"], rng["end"], rng["step"])]
    tasks = [{**cmd , "load" : load} for load in loads]
    executor = ProcessPoolExecutor(30)
    futures = [executor.submit(run_nox, task) for task in tasks]

    #show progree bar tqdm
    for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
        res = future.result()
        results.append([res[0]["load"] , res[1]])
    return sorted(results, key=lambda x: x[0])


In [18]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

routing = Routing.XY
traffic = Traffic.RANDOM
rng = { "start": 0.01 , "end" : 1 , "step" : 0.01}
size = [4,4]
cmd = make_cmd(load=0 , size=size, routing = routing , traffic = traffic)
buffer_sizes = [2,4,8,16,32,64]

results = []

for buffer_size in buffer_sizes:
    result = runOnRange_par({**cmd , 'buffer' : buffer_size } ,rng)
    results.append([buffer_size , result])

# Creating subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=('Latency', 'Throughput'))

# Adding traces for latency and throughput
for result in results:
    loads = [x[0] for x in result[1]]
    latency = [x[1]['average_delay'] for x in result[1]]
    throughput = [x[1]['network_throughput'] for x in result[1]]
    
    # Latency plot
    fig.add_trace(
        go.Scatter(x=loads, y=latency, mode='lines+markers', name=f'buffer size {result[0]}'),
        row=1, col=1
    )

    # Throughput plot
    fig.add_trace(
        go.Scatter(x=loads, y=throughput, mode='lines+markers', name=f'buffer size {result[0]}'),
        row=1, col=2
    )

# Update layout
fig.update_layout(title_text=f'{routing.name} Routing, {size[0]}*{size[1]} Mesh, {traffic.name} Traffic', 
                  width=1000, height=500)

fig.show()


100%|██████████| 99/99 [00:10<00:00,  9.16it/s]
